In [ ]:
# とりあえず動くものを実装する
from pathlib import Path
import asyncio
import pickle
import datetime
import json

import numpy as np
from pydantic import BaseModel
import polars as pl

import stock
import data_fetcher
import auto_trader

In [ ]:
model_root_dir = Path("/home/kitamura/work/stock/outputs/")
model_path = model_root_dir / "20241123_crypto_train" / "model_2024-11-24T18:32:34.455716.pkl"
log_dir = auto_trader.constants.PROJECT_ROOT / "log"

In [ ]:
trader = auto_trader.trader.Trader0(
    symbol="BTC_JPY",
    interval=datetime.timedelta(minutes=1),
    model_path=model_path,
    log_dir=log_dir,
)

In [ ]:
try: 
    trader.run_loop()
except Exception as e:
    trader.cancel_all_orders()
    raise e

In [ ]:
# モデルの問題点を検証

In [ ]:
symbol = "BTC_JPY"
interval_minutes = 1
train_ratio = 0.7
pips = 0.01

max_loss_rate = 0.1
max_hold_timestep = 10

fetcher = data_fetcher.gmo.GMOFethcer()
all_df = fetcher.fetch_ohlc(symbol, interval=datetime.timedelta(minutes=interval_minutes))

In [ ]:
train_num = (int)(len(all_df) * train_ratio)
train_df = all_df[:train_num]
test_df = all_df[train_num:]

train_df = stock.crypto.feature.calc_features(train_df).filter(
    pl.all_horizontal(pl.col(pl.Float32, pl.Float64).is_not_nan())
)
test_df = stock.crypto.feature.calc_features(test_df).filter(
    pl.all_horizontal(pl.col(pl.Float32, pl.Float64).is_not_nan())
)

In [ ]:
y_test = trader.model.predict(test_df.select(*auto_trader.trader.trader0.train_features))

In [ ]:
# simulation
results = []
result = []
for i in range(len(test_df) - 1):
    if y_test[i] <= 0:
        continue

    target_buy_price = test_df["close"][i] - test_df["ATR"][i] * 0.8
    target_sell_price = test_df["close"][i] + test_df["ATR"][i] * 0.8
    if len(result) > 0:
        if target_sell_price < test_df["high"][i + 1]:
            result += [i + 1, target_sell_price]
            results.append(result)
            result = []
    else:
        if target_buy_price > test_df["low"][i + 1]:
            result += [i + 1, target_buy_price]

In [ ]:
results = np.array(results)

In [ ]:
(results[:, 3] - results[:, 1]).sum()